In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# own Modules 
from models import LstmMseDropout
from data_loader import DataPreperator, DataSet
from trainer import Trainer
from loss_module import LossMse, LossMle

## Hyperparameters

In [5]:
hyperparam = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../../data/artifical_signals/NewBlade_with_sine.csv',
        "droped_feature" : ["timestamp"]
    },
    "model" : {
        "input_size" : 8,
        "n_hidden_lstm" : [50, 100, 200, 300],
        "sequence_size" : [20, 50, 150, 300],
        "batch_size" : 8,
        "lstm_layer" : [2, 4],
        "n_hidden_fc": [75, 125],
        "dropout_rate": 0.2
        
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # step_size is the number of training iterations (total samples/batch_size) per half cycle. 
        # Authors suggest setting step_size 2-8 x training iterations in epoch.
        "step_size" : (2048/8)*2,
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular",
        "gamma" : 0.9995,
        "base_lr" : 0.016, 
        "max_lr" :0.1
    },
    "training": {
        "n_epochs" : 100,
        "patience" : 10,
    },
    "filed_location": {
        "trained_model" : "../../../models/MSE_model/grid_search/best_model",
        "statistics" : "../../visualisation/files/history_training/grid_search/statistics_grid_search.csv"
    }
}

## Split raw data into train and validation data and scale it

In [6]:
train_loader = DataPreperator(path=hyperparam['data']['path'], 
                              stake_training_data=hyperparam['data']['stake_training_data'],
                              ignored_features=hyperparam['data']['droped_feature'])
train_data, validation_data = train_loader.prepare_data()

## Grid Search
An epoch consists of a learning cycle over all batches of training data and an evaluation of the most recent model with the testing data. 

In [4]:
for n_lstm_layer in hyperparam["model"]["lstm_layer"]:
    for sequence_size  in hyperparam["model"]["sequence_size"]:
        # Create roling dataset 
        dataset_train = DataSet(train_data, timesteps=sequence_size)
        dataset_validation = DataSet(validation_data, timesteps=sequence_size)
    
        # Initialize DataLoader
        data_loader_training = DataLoader(dataset_train, batch_size=hyperparam["model"]["batch_size"], 
                                          num_workers=1, shuffle=True, drop_last=True)
        data_loader_validation = DataLoader(dataset_validation, batch_size=hyperparam["model"]["batch_size"], 
                                            num_workers=1, shuffle=True, drop_last=True)
        for n_hidden_lstm in hyperparam["model"]["n_hidden_lstm"]:
            for n_hidden_fc in hyperparam["model"]["n_hidden_fc"]:
                print("Start with new hyperparameters in grid search: ")
                print("Sequence_size: {}".format(sequence_size))
                print("Number LSTM Layers: {}".format(n_lstm_layer))
                print("LSTM Number Hidden Dimensions: {}".format(n_hidden_lstm))
                print("FC NN Number Hidden Dimensions: {}".format(n_hidden_fc))

                # Create lists to save training loss and validation loss of each epoch
                hist_loss = []
                torch.manual_seed(0)
                model = LstmMseDropout(batch_size=hyperparam['model']['batch_size'], 
                                input_dim=hyperparam['model']['input_size'], 
                                n_hidden_lstm=n_hidden_lstm, 
                                n_layers=n_lstm_layer,
                                dropout_rate= hyperparam['model']['dropout_rate'],
                                n_hidden_fc=n_hidden_fc
                               )

                optimizer = torch.optim.SGD(model.parameters(), lr=1.)  

                criterion = LossMse(hyperparam["model"]["input_size"], hyperparam["model"]["batch_size"])

                scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                                              base_lr=hyperparam['cycling_lr']['base_lr'], 
                                                              max_lr=hyperparam['cycling_lr']['max_lr'], 
                                                              step_size_up=hyperparam['cycling_lr']['step_size'], 
                                                              mode=hyperparam['cycling_lr']['mode'],
                                                              gamma=hyperparam['cycling_lr']['gamma']
                                                             )

                trainer = Trainer(model=model,
                                  optimizer=optimizer,
                                  scheduler=scheduler,
                                  scheduler_active = param["cycling_lr"]["scheduler_active"],
                                  criterion=criterion, 
                                  location_model=hyperparam["filed_location"]["trained_model"], 
                                  location_stats=hyperparam["filed_location"]["statistics"], 
                                  patience=hyperparam['training']['patience']
                                 )
                
                # Measure training time for current configuration
                start = time.time()

                for epoch in range(hyperparam['training']['n_epochs']):
                    # Train
                    mean_epoch_training_loss = trainer.train(data_loader_training)

                    # Evaluate
                    mean_epoch_validation_loss = trainer.evaluate(data_loader_validation, hist_loss, epoch)

                    # Cache History
                    trainer.cache_history_val_loss(hist_loss, epoch, mean_epoch_validation_loss)

                    # Save model if its the best one since the last change in configuration of hyperparameters
                    status_ok = trainer.save_model(epoch, mean_epoch_validation_loss, hyperparam['model']['input_size'], 
                                                   n_lstm_layer, n_hidden_lstm, n_hidden_fc, sequence_size)
                    if not status_ok:
                        break

                # Time in minutes
                execution_time = (time.time() - start)/60
                
                # Save training statistics 
                trainer.save_statistic(hist_loss, sequence_size, n_lstm_layer, n_hidden_lstm, n_hidden_fc, execution_time)         

Start with new hyperparameters in grid search: 
Sequence_size: 20
Number LSTM Layers: 2
LSTM Number Hidden Dimensions: 50
FC NN Number Hidden Dimensions: 75


/Users/carlovoss/jupyter_notebooks/masterarbeit/venv_pm/lib/python3.6/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


-------- epoch_no. 0 finished with eval loss 0.9608350654443105--------
Epoch 0: best model saved with loss: 0.9608350654443105
-------- epoch_no. 1 finished with eval loss 0.8323171059290568--------
Epoch 1: best model saved with loss: 0.8323171059290568
-------- epoch_no. 2 finished with eval loss 0.6608158320188522--------
Epoch 2: best model saved with loss: 0.6608158320188522
-------- epoch_no. 3 finished with eval loss 0.6764779220024745--------


Traceback (most recent call last):
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [2]:
def dule():
    global a
    a = 5
dule()
print(a)

5
